# 🔍 MCP Server Explorer

This notebook provides an interactive environment to explore and test tools exposed by an MCP (Model Context Protocol) server using the `fastmcp` and `fast-agent-mcp` libraries.

### ✅ Features
- **🔧 Fetch Tools**: Connect to a remote MCP server and list all available tools
- **⚙️ Call Tool Directly**: Invoke a tool programmatically with structured input
- **🧠 Call Tool via LLM**: Let a language model select and execute tools automatically using `FastAgent`

> Ideal for debugging, prototyping, and understanding how tools behave in both manual and AI-driven workflows.


## 📦 Installation

Install all required packages:
- fast-agent-mcp – Agent orchestration framework
- fastmcp – MCP client for tool/resource interaction
- pandas – Data handling and visualization
- python-dotenv – Load environment variables from .env files
- asyncio – Built-in async support (included for completeness)

In [ ]:
!pip install fast-agent-mcp fastmcp pandas python-dotenv asyncio nest_asyncio

import os
from mcp_agent.core.fastagent import FastAgent
from fastmcp import Client
from IPython.display import display, Markdown
from pprint import pprint
import pandas as pd
import argparse
import asyncio
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()

## 🔐 Load Environment Variables from `.env` Files

The `load_dotenv()` function from the `python-dotenv` package reads key-value pairs from a `.env` file and adds them to `os.environ`, making them available as environment variables in your Python code.


In [ ]:
load_dotenv()

## 🧰 Helper Functions

This section contains utility functions that simplify or standardize common tasks throughout the notebook. These helpers improve code readability, reduce duplication, and make the notebook easier to maintain.

### ✅ Guidelines
- Keep each function focused on a single responsibility
- Reuse helpers wherever possible (e.g., formatting, conversion, API response handling)
- Document inputs/outputs with comments or docstrings

In [ ]:
class NotebookAgent(FastAgent):
    """
    A customized FastAgent subclass for Jupyter notebook environments.

    This class disables default CLI interaction output (e.g., agent messages like
    [USER], [ASSISTANT]) by faking CLI arguments using `argparse.Namespace`.

    Ideal for use in notebooks where minimal stdout noise is preferred.
    """

    def __init__(self):
        """
        Initialize the NotebookAgent with quiet mode enabled.

        Sets `self.args.quiet = True` to suppress verbose output,
        and names the agent 'NotebookAgent' for identification.
        """
        super().__init__("NotebookAgent")
        self.args = argparse.Namespace()
        self.args.quiet = True

def tool_to_dict(tool):
    """
    Convert a tool object into a simplified dictionary representation.

    This helper extracts key attributes from a tool, including metadata,
    descriptive hints, and input schema properties, for easier inspection
    or display in tabular or JSON formats.

    Parameters:
        tool (Tool): A tool object (e.g., from fastmcp or fast-agent-mcp)
                     with properties like name, description, annotations, and inputSchema.

    Returns:
        dict: A dictionary with the following keys:
            - name (str): Tool's internal name.
            - title (str or None): Optional display title from annotations.
            - description (str): Tool's description.
            - readOnly (bool or None): Hint whether the tool is read-only.
            - destructive (bool or None): Hint whether the tool has side effects.
            - inputs (list of str): Names of input fields defined in the input schema.
    """
    return {
        "name": tool.name,
        "title": tool.annotations.title if tool.annotations else None,
        "description": tool.description,
        "readOnly": tool.annotations.readOnlyHint if tool.annotations else None,
        "destructive": tool.annotations.destructiveHint if tool.annotations else None,
        "inputs": list(tool.inputSchema.get("properties", {}).keys())
    }


## 🤖 Call Claude via FastAgent

This cell defines and runs a simple `FastAgent` using Claude (or another LLM configured via `fast-agent-mcp`). The agent receives an instruction to act as a helpful assistant and responds to a natural language question.

- The agent is declared with `@fast.agent(...)`
- The `fast.run()` context initializes and executes the agent
- A sample query (`"What is the capital of Germany?"`) is sent
- The response is printed directly

> 💡 You can replace the instruction, prompt, or model by adjusting the decorator or `FastAgent` configuration.


In [ ]:
fast = NotebookAgent()

@fast.agent(instruction="You are a helpful assistant.")
async def run():
    async with fast.run() as agent:
        response = await agent("What is the capital of Germany?")
        print(response)

asyncio.run(run())

## 🤖 Call OpenAI (GPT-4o) via FastAgent

This cell runs an OpenAI-powered assistant using `fast-agent-mcp` and a custom `NotebookAgent` to suppress noisy CLI-style output in Jupyter.

- The agent is configured with `model="gpt-4o"` to use OpenAI
- Instruction: `"You are a helpful assistant."`
- Prompt: `"Explain black holes in one sentence."`
- The response is printed directly in the notebook

> ✅ This setup uses `NotebookAgent` to keep the notebook output clean and readable while leveraging the OpenAI API.


In [ ]:
fast = NotebookAgent()

# Use model="gpt-4o" to switch to OpenAI
@fast.agent(instruction="You are a helpful assistant.", model="gpt-4o")
async def run():
    async with fast.run() as agent:
        response = await agent("Explain black holes in one sentence.")
        print(response)

asyncio.run(run())

## 🌐 FastAgent with OpenAI + Web Search (MCP Tool)

This cell sets up an OpenAI-based agent that can use external tools — in this case, the `brave_websearch` MCP server — to answer questions with real-time search data.

#### 🧠 What It Does:
- Instantiates a `NotebookAgent` (a quiet `FastAgent` variant for clean notebook output)
- Registers an agent named `"assistant"` with:
  - Instruction: *"You are a helpful assistant."*
  - Model: `gpt-4o` (OpenAI)
  - Tool access via `servers=["brave_websearch"]` (MCP tool server that can perform web searches)
- The LLM decides to call the `brave_websearch` tool to answer:
  > "At which position is eurowings.com ranked on Google for 'Mallorca'?"

#### 🔧 Key Concepts:
- ✅ **Tool Calling via LLM**: The assistant can dynamically decide to call the `brave_websearch` tool if it needs up-to-date info.
- ✅ **Hybrid Reasoning**: Combines LLM reasoning with external tool execution.

> 🧪 Ideal for agents that require real-time or external knowledge beyond the LLM’s static training data.


In [ ]:
fast = NotebookAgent()

@fast.agent(name="assistant",
            instruction="You are a helpful assistant.",
            model="gpt-4o",
            servers=["brave_websearch"])
async def run():
    async with fast.run() as agent:
        response = await agent("At which position is eurowings.com ranked on google for 'Mallorca'")
        print(response)

asyncio.run(run())

## ⚙️ Use Playwright via `fastmcp.Client`

This cell demonstrates how to connect to a local MCP server (in this case, `@playwright/mcp`) using the `fastmcp` client and display its available tools.

#### 🔧 What It Does:
- Defines an MCP client configuration (`config`) that starts the `@playwright/mcp` tool server using `npx`
- Creates a `Client(config)` to interface with that MCP server
- Uses an async context manager (`async with client`) to:
  - Fetch a list of available tools from the `playwright` MCP server
  - Convert each tool into a dictionary using `tool_to_dict(...)`
  - Display the tools in a structured table using `pandas`

In [ ]:
config = {
    "mcpServers": {
        "playwright": {
            "command": "npx",
            "args": ["-y", "@playwright/mcp@latest"]
        }
    }
}

client = Client(config)

async def run():
    async with client:
        mcp_tools = await client.list_tools()
        df = pd.DataFrame([tool_to_dict(t) for t in mcp_tools])
        display(df)

asyncio.run(run())

## 🌐 Website Navigation with Playwright MCP Tool

This cell configures and uses the `@playwright/mcp` server (launched via `npx`) to simulate browser navigation to a specified URL using the `fastmcp` client.


In [ ]:
config = {
    "mcpServers": {
        "playwright": {
            "command": "npx",
            "args": ["-y", "@playwright/mcp@latest"],
        }
    }
}

client = Client(config)

async def run():
    async with client:
        response = await client.call_tool("browser_navigate", {"url": "https://github.com/messeb"})
        # pprint(response[0].text)
asyncio.run(run())

## 🌐 Use Brave Search via MCP Tool via `fastmcp.Client`

This cell demonstrates how to integrate and use the `@modelcontextprotocol/server-brave-search` MCP tool server to:
1. List all available tools (i.e., tool discovery)
2. Perform a live web search using the Brave Search API

In [ ]:
config = {
    "mcpServers": {
        "brave_websearch": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-brave-search"],
            "env": {
                "BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")
            }
        }
    }
}

client = Client(config)

async def run_list():
    async with client:
        mcp_tools = await client.list_tools()
        df = pd.DataFrame([tool_to_dict(t) for t in mcp_tools])
        display(df)

async def run_tool():
    async with client:
        response = await client.call_tool("brave_web_search", {"query": "Mallorca", "count": 5, "offset": 0})
        display(Markdown(response[0].text))

asyncio.run(run_list())
asyncio.run(run_tool())
